In [1]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.7 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [4]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [6]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [7]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [8]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [9]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 46s]
val_accuracy: 0.8532500267028809

Best val_accuracy So Far: 0.8917499780654907
Total elapsed time: 00h 24m 54s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 256 and the optimal learning rate for the optimizer
is 0.001.



In [10]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 11s 7ms/step - loss: 0.5059 - accuracy: 0.8208 - val_loss: 0.3851 - val_accuracy: 0.8604
Epoch 2/50
1500/1500 [==============================] - 10s 6ms/step - loss: 0.3767 - accuracy: 0.8643 - val_loss: 0.3887 - val_accuracy: 0.8622
Epoch 3/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3345 - accuracy: 0.8770 - val_loss: 0.3461 - val_accuracy: 0.8729
Epoch 4/50
1500/1500 [==============================] - 10s 7ms/step - loss: 0.3099 - accuracy: 0.8845 - val_loss: 0.3490 - val_accuracy: 0.8754
Epoch 5/50
1500/1500 [==============================] - 10s 6ms/step - loss: 0.2912 - accuracy: 0.8922 - val_loss: 0.3264 - val_accuracy: 0.8845
Epoch 6/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2766 - accuracy: 0.8961 - val_loss: 0.3316 - val_accuracy: 0.8800
Epoch 7/50
1500/1500 [==============================] - 10s 7ms/step - loss: 0.2637 - accuracy: 0.9016 - val_loss: 0.3313 - val_accu

In [11]:
hypermodel = tuner.hypermodel.build(best_hps)

hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/26
1500/1500 [==============================] - 10s 7ms/step - loss: 0.5006 - accuracy: 0.8229 - val_loss: 0.4230 - val_accuracy: 0.8483
Epoch 2/26
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3747 - accuracy: 0.8633 - val_loss: 0.4157 - val_accuracy: 0.8484
Epoch 3/26
1500/1500 [==============================] - 10s 6ms/step - loss: 0.3329 - accuracy: 0.8773 - val_loss: 0.3323 - val_accuracy: 0.8837
Epoch 4/26
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3105 - accuracy: 0.8861 - val_loss: 0.3323 - val_accuracy: 0.8813
Epoch 5/26
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2882 - accuracy: 0.8945 - val_loss: 0.3241 - val_accuracy: 0.8814
Epoch 6/26
1500/1500 [==============================] - 10s 7ms/step - loss: 0.2739 - accuracy: 0.8979 - val_loss: 0.3242 - val_accuracy: 0.8827
Epoch 7/26
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2623 - accuracy: 0.9027 - val_loss: 0.3266 - val_accura

In [12]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 3ms/step - loss: 0.4517 - accuracy: 0.8819
[test loss, test accuracy]: [0.45173704624176025, 0.8819000124931335]
